In [ ]:
from fugle_realtime_api import *

In [ ]:
api_token = 'YOUR FUGLE API TOKEN'
line_token = 'YOUR LINE NOTIFY TOKEN'

In [ ]:
line = notify_setting(api_token=api_token,line_token=line_token)

In [ ]:
fugle_chart = plot_chart_data(api_token=api_token, symbol_id='2884')
fugle_trades = plot_trades_data(api_token=api_token, symbol_id='2884')

In [ ]:
fugle_quote = plot_quote_data(api_token=api_token, symbol_id='2884')
fugle_quote.get_first_order_book()

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

#If you want to know mroe html language in Dash, please visit this url -> https://dash.plot.ly/dash-html-components

#Dash apps store the app.layout in memory.
app.layout = html.Div(children=[
    
    html.Div(
        id='min_K_volume',
        #height：means the height of th graph
        #width：means the width of th graph
        #display：'inline-block' means merge two graph on the same line
        style={
            'width': '54%',
            'display': 'inline-block',
            'vertical-align': 'top'
        }),
    
    html.Div([
        dcc.Graph(id='price_vol_content', style={'height':'50vh'}),
        #line_strategy_bottom means that we can create switch of trading strategy.
        line.line_strategy_bottom(id_name='switch1', label_name='strategy-1'),
        line.line_strategy_bottom(id_name='switch2',label_name='strategy-2'),
        line.line_strategy_bottom(id_name='switch3',label_name='strategy-3')],
        
        style={'width': '20%','display': 'inline-block','vertical-align': 'top'}),
    
    html.Div(id='best5_table',style={'width': '25%','display': 'inline-block'}),

    #interval property means that you can set the update interval of data -> in milliseconds
    #n_intervals property is an integer that is automatically incremented every time interval milliseconds pass.
    dcc.Interval(id='interval-component', interval=5000, n_intervals=1),
    html.Div(id='nothing')
])


@app.callback(Output('min_K_volume', 'children'),
              [Input('interval-component', 'n_intervals')])
def min_K_plot(interval_component):

    return html.Div([
        dcc.Graph(
            id='min_K',
            figure={
                'data': [
                    fugle_chart.min_K(n=5,up_color='red',down_color='green'),
                    fugle_trades.avg_cost_line(n=5,line_color='orange')
                ],
                'layout': {
                    # 'showticklabels:False' means unable to display xticks.
                    # 'rangeslider': {'visible':False} means unable to display timeslider under xaxis.
                    #'title' means that you can set the title of graph
                    'xaxis': { 'rangeslider': {'visible': False}, 'showticklabels': False},
                    'title': '2884 5-minK',
                    'margin': { 'b': 0,'l': 50, 'r': 0},
                    'legend': {'orientation': 'h'}
                }
            },
            style = {'height':'50vh'}
        ),
        
        dcc.Graph(
            id='min_volume',
            figure = {
                'data': [fugle_chart.min_volume(up_color='red', down_color='green')],
                
                'layout': {
                    'margin': { 't': 0,'b': 70,'r': 0,'l': 50}
                }
            },
            style = {'height':'30vh'}
        )
    ])


@app.callback(Output('price_vol_content', 'figure'),
              [Input('interval-component', 'n_intervals')])
def price_vol_plot(interval_component):

    return {
        'data': [fugle_trades.volume_of_price()],
        'layout': {'margin': {'b': 0,'r': 0}}
    }


@app.callback(Output('best5_table', 'children'),
              [Input('interval-component', 'n_intervals')])
def order_book_plot(interval_component):

    fugle_quote.update_order_book()

    return html.Div([
        fugle_quote.order_book(header_fontsize=16,cell_fontsize=14)
    ])


@app.callback(Output('nothing', 'children'), [
    Input('interval-component', 'n_intervals'),
    Input('switch1', 'on'),
    Input('switch2', 'on'),
    Input('switch3', 'on'),
])
def interact_strategy(interval_component, switch1, switch2, switch3):
    #switch == True means that open the trading strategy reminder
    if switch1 == True:

        line.price_strategy(symbol_id='2884', up_price=28, down_price=27)

    if switch2 == True:

        line.price_change_strategy(symbol_id='2884',
                                   up_rate=0.02,
                                   down_rate=0.02)

    if switch3 == True:

        line.price_strategy(symbol_id='2884', up_price=28, down_price=27)

    else:

        pass


if __name__ == '__main__':
    app.run_server(host='0.0.0.0')